In [1]:
import keras
import numpy as np
import scipy.io
from pathlib import Path
import os

Using TensorFlow backend.


## Open labels

In [2]:
path = 'C:\\Users\\Rodolfo\\Documents\\algoritmos-git\\custom_cnn\\Data\\Processados\\'

In [3]:
y_train = np.load(Path(path,'y_train.npy'))
y_test = np.load(Path(path, 'y_test.npy'))
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [4]:
from keras.utils import np_utils
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Open data

In [7]:
path = Path(path, '28_28')
X_train_1 = np.load(Path(path, 'X_train_1.npy'))
X_train_2 = np.load(Path(path, 'X_train_2.npy'))
X_test_1 = np.load(Path(path, 'X_test_1.npy'))
X_test_2 = np.load(Path(path, 'X_test_2.npy'))
print(X_train_1.shape)
print(X_train_2.shape)
print(X_test_1.shape)
print(X_test_2.shape)

(2200, 28, 28)
(2200, 28, 28)
(1000, 28, 28)
(1000, 28, 28)


### Shape desejado: x_train shape: (45000, 32, 32, 3) -> n_obs, n_row, n_col, n_chan

In [8]:
n_obs, n_row, n_col = X_train_1.shape
n_chan=2
X_train = np.zeros((n_obs, n_row, n_col, n_chan))
X_train.shape

(2200, 28, 28, 2)

In [9]:
X_train[:,:,:,0]=X_train_1
X_train[:,:,:,1]=X_train_2
X_train.shape

(2200, 28, 28, 2)

In [10]:
n_obs, n_row, n_col = X_test_1.shape
n_chan=2
X_test = np.zeros((n_obs, n_row, n_col, n_chan))
X_test.shape

(1000, 28, 28, 2)

In [11]:
X_test[:,:,:,0]=X_test_1
X_test[:,:,:,1]=X_test_2
X_test.shape

(1000, 28, 28, 2)

# Shuffle X_train

In [12]:
arr = np.arange(y_train.shape[0])
np.random.shuffle(arr)
X_train = X_train[arr,:,:,:]
y_train = y_train[arr]

### 3. Rescale the Images by Dividing Every Pixel in Every Image by 255

In [13]:
# rescale [0,255] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### 4.  Break Dataset into Training, Testing, and Validation Sets

In [14]:
# break training set into training and validation sets
(X_train, X_valid) = X_train[:1800], X_train[1800:]
(y_train, y_valid) = y_train[:1800], y_train[1800:]

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (1800, 28, 28, 2)
1800 train samples
1000 test samples
400 validation samples


In [15]:
n_obs, n_row, n_col, n_chan = X_train.shape

In [16]:
print(n_obs, n_row, n_col, n_chan)

1800 28 28 2


### 5. Define the Model Architecture 

In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# "valid" means no padding. "same" results in padding
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='valid', activation='relu', strides=(1,1), input_shape=(n_row, n_col, n_chan)))
model.add(MaxPooling2D(pool_size=2, strides=(1,1)))
model.add(Conv2D(filters=32, kernel_size=2, padding='valid', activation='relu', strides=(2,2)))
model.add(MaxPooling2D(pool_size=2))

'''
model.add(Conv2D(filters=64, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='valid', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
'''
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 27, 16)        144       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
______________________

### 6. Compile the Model 

In [18]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

### 7. Train the Model 

In [19]:
model_path = 'model_13.weights.best.hdf5'

In [20]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=100,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer], 
          verbose=2, shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1800 samples, validate on 400 samples
Epoch 1/100





 - 3s - loss: 0.6988 - acc: 0.5194 - val_loss: 0.7067 - val_acc: 0.5075

Epoch 00001: val_loss improved from inf to 0.70673, saving model to model_13.weights.best.hdf5
Epoch 2/100
 - 2s - loss: 0.6797 - acc: 0.5778 - val_loss: 0.7228 - val_acc: 0.5000

Epoch 00002: val_loss did not improve from 0.70673
Epoch 3/100
 - 2s - loss: 0.6642 - acc: 0.6006 - val_loss: 0.7683 - val_acc: 0.5125

Epoch 00003: val_loss did not improve from 0.70673
Epoch 4/100
 - 2s - loss: 0.6571 - acc: 0.6194 - val_loss: 0.6743 - val_acc: 0.5825

Epoch 00004: val_loss improved from 0.70673 to 0.67427, saving model to model_13.weights.best.hdf5
Epoch 5/100
 - 2s - loss: 0.6367 - acc: 0.6389 - val_loss: 0.6816 - val_acc: 0.5525

Epoch 00005: val_loss did not improve from 0.67427
Epoch 6/100
 - 2s - loss: 0.6211 - acc: 0.6528 - val_loss: 0.6426 - val_acc: 0.


Epoch 00044: val_loss did not improve from 0.60803
Epoch 45/100
 - 2s - loss: 0.2667 - acc: 0.8906 - val_loss: 0.6770 - val_acc: 0.6550

Epoch 00045: val_loss did not improve from 0.60803
Epoch 46/100
 - 2s - loss: 0.2731 - acc: 0.8844 - val_loss: 0.8136 - val_acc: 0.6675

Epoch 00046: val_loss did not improve from 0.60803
Epoch 47/100
 - 5s - loss: 0.2644 - acc: 0.8939 - val_loss: 0.7855 - val_acc: 0.6500

Epoch 00047: val_loss did not improve from 0.60803
Epoch 48/100
 - 6s - loss: 0.2543 - acc: 0.8956 - val_loss: 0.7454 - val_acc: 0.6800

Epoch 00048: val_loss did not improve from 0.60803
Epoch 49/100
 - 2s - loss: 0.2493 - acc: 0.9072 - val_loss: 0.7237 - val_acc: 0.6325

Epoch 00049: val_loss did not improve from 0.60803
Epoch 50/100
 - 2s - loss: 0.2606 - acc: 0.8911 - val_loss: 0.7737 - val_acc: 0.6450

Epoch 00050: val_loss did not improve from 0.60803
Epoch 51/100
 - 2s - loss: 0.2351 - acc: 0.9111 - val_loss: 0.7853 - val_acc: 0.6075

Epoch 00051: val_loss did not improve fr

In [21]:
model_path

'model_13.weights.best.hdf5'

### 8. Load the Model with the Best Validation Accuracy

In [22]:
# load the weights that yielded the best validation accuracy
model.load_weights(model_path)

### 9. Calculate Classification Accuracy on Test Set

In [23]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.722


In [24]:
model.predict(X_test)

array([[0.28381956, 0.7161804 ],
       [0.94627106, 0.0537289 ],
       [0.70623875, 0.29376128],
       ...,
       [0.12042034, 0.8795797 ],
       [0.7401453 , 0.25985464],
       [0.22487183, 0.7751282 ]], dtype=float32)

In [ ]:
model_path